In [1]:
import time

In [2]:
# pour un ordi avec GPU :

import GPUtil
import torch

In [3]:
torch.cuda.is_available()

True

In [5]:
!set CUDA_PATH=C:\Users\ehess\anaconda3\Lib\site-packages\numba\cuda

import spacy

In [8]:
# spacy.require_gpu()
# spacy.prefer_gpu()
#  j'utilise thinc.api

In [6]:
# pour un ordi avec GPU
# j'utilise thinc

# is_using_gpu = spacy.prefer_gpu()
# if is_using_gpu:
#     print("Using GPU!")
#     torch.set_default_tensor_type("torch.cuda.FloatTensor")
#     print("GPU Usage")
GPUtil.showUtilization()

| ID | GPU | MEM |
------------------
|  0 |  0% |  8% |


In [7]:
import pandas as pd

In [8]:
df = pd.read_csv("contrib_from_csv.csv", sep = ",", encoding = "utf-8", dtype= str)

# sample d'un petit échantillon
# df = df.sample(1000)

In [9]:
# supprimer les fin de lignes (deux types de codages)
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 

# supprimer les UNK des ce corpus (trois types de codages)
kill_texte_illisible = lambda x : x.replace("texte illisible", "")
kill_illisible = lambda x : x.replace("illisible", "")
kill_illisible_plural = lambda x : x.replace("illisibles", "")
kill_illisible_capslock = lambda x : x.replace("ILLISIBLE", "")
kill_illisibleS_capslock = lambda x : x.replace("ILLISIBLES", "")

In [10]:
df["clean"] = df["Contribution"].apply(join_breaklines)
df["clean"] = df["clean"].apply(join_breaklinesn)
df["clean"] = df["clean"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)
df["clean"] = df["clean"].apply(kill_illisible_capslock)
df["clean"] = df["clean"].apply(kill_illisibleS_capslock)

In [11]:
from thinc.api import set_gpu_allocator, require_gpu, prefer_gpu, use_pytorch_for_gpu_memory, set_active_gpu
set_gpu_allocator("pytorch")
import csv   

def lemmatize(doc, subset):
    '''
    Performs lemmization of input documents.
    Args:
    - docs: one column of a dataframe
    Output:
    - list (nb of contributions) of list (one cell,(size 2: with position input, lemmatized input)) of list of strings
    '''

    
    set_gpu_allocator("pytorch")
    require_gpu()
    set_active_gpu(0)
    if prefer_gpu():
        print("Using GPU!")
        torch.set_default_tensor_type("torch.cuda.FloatTensor")
        use_pytorch_for_gpu_memory()

        print("GPU Usage")
        GPUtil.showUtilization()

    nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])

    spacy.require_gpu()

    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.

    t0 = time.time()
    
    len_data = doc.shape[0]
    nb_batch = len_data // subset
    size_last_batch = len_data%subset
    first_idx = 0
    last_idx = 0
        
    for batch_nb in range(nb_batch) :
        t1 = time.time()
        lemma_text_list = []
        lemma_pos_list = []
        
        if batch_nb == nb_batch -1 :
            last_idx += size_last_batch
        else :
            first_idx = last_idx
            last_idx += subset
        
        batch = doc[first_idx : last_idx]
        
        for line in nlp.pipe(batch):
#             ta = time.time()
            lemma_pos_list.append([x.pos_ for x in line])
#             GPUtil.showUtilization()

            lemma_text_list.append([x.lemma_ for x in line])
#             tc = time.time()
#             print(f"spacy computing time per line: {tc-ta}")

        print("GPU Usage")
        GPUtil.showUtilization()
        
        with open(f'output_lemmas/lemma_temp_{batch_nb}.csv', 'w+', encoding = "utf-8") as f :
            writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL, escapechar = '\\')
            writer.writerow(lemma_text_list)
            f.close()
            
        with open(f'output_pos/pos_temp_{batch_nb}.csv', 'w+', encoding = "utf-8") as f :
            writer = csv.writer(f, delimiter=',', quoting=csv.QUOTE_MINIMAL, escapechar = '\\')
            writer.writerow(lemma_pos_list)
            f.close()

        
        del lemma_pos_list
        del lemma_text_list
#     for doc in nlp.pipe(doc):
#         buffer = []
#         lemma_text_list.append([[" ".join(x.pos_ for x in doc)], [" ".join(x.lemma_ for x in doc)]])     
#        on ne join() pas à ce moment là car la list reste nécessaire pour enlever les stopwords.

        t2 = time.time()
        print(f"end of batch: {batch_nb}, time: {t2-t1}")
        
    t3 = time.time()
    print("Total time: {}".format(t3-t0))

In [12]:
# GPU Total time: 130.2461621761322 (without if and with fr_core_news_md)
lemmatize(df["clean"], 64)

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 17% | 16% |
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  5% | 27% |
1
2
end of batch: 0, time: 3.1547508239746094
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 30% |
1
2
end of batch: 1, time: 2.1496448516845703
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  3% | 30% |
1
2
end of batch: 2, time: 1.9666938781738281
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
1
2
end of batch: 3, time: 2.1614787578582764
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
1
2
end of batch: 4, time: 1.437446117401123
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
1
2
end of batch: 5, time: 1.5603082180023193
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
1
2
end of batch: 6, time: 1.4073207378387451
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 34% |
1
2
end of batch: 7, time: 1.0726454257965088
GPU Usage
| ID | GP

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 72, time: 0.7777857780456543
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 73, time: 0.7502415180206299
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 74, time: 0.9997560977935791
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 75, time: 1.0468122959136963
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 76, time: 0.7881872653961182
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 77, time: 1.344564437866211
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 78, time: 0.9880051612854004
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 79, time: 1.407520055770874
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 80, t

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 144, time: 0.8404765129089355
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 145, time: 0.6485488414764404
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 146, time: 0.7419896125793457
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 147, time: 0.764704704284668
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 148, time: 1.1120340824127197
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 149, time: 0.9888818264007568
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 150, time: 0.6563565731048584
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 151, time: 0.9021282196044922
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of bat

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 216, time: 0.49610471725463867
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 217, time: 1.0157930850982666
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 218, time: 1.4549267292022705
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 219, time: 0.7487716674804688
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 220, time: 0.9044010639190674
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 221, time: 0.7767930030822754
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 222, time: 1.1397366523742676
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of batch: 223, time: 0.981008768081665
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 44% |
1
2
end of ba

GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 38% | 49% |
1
2
end of batch: 288, time: 0.4908435344696045
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 289, time: 0.6108877658843994
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 290, time: 0.7867465019226074
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 291, time: 0.7823057174682617
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 292, time: 1.2070584297180176
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 293, time: 1.4659192562103271
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 294, time: 0.9177694320678711
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of batch: 295, time: 0.6985428333282471
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 49% |
1
2
end of ba

...

GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 41% | 55% |
1
2
end of batch: 504, time: 0.5468099117279053
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 505, time: 0.7656004428863525
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 506, time: 0.9843106269836426
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 507, time: 0.6249294281005859
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 508, time: 0.8124449253082275
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 509, time: 0.9687044620513916
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 510, time: 0.71871018409729
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch: 511, time: 0.59371018409729
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 55% |
1
2
end of batch:

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 576, time: 0.7030766010284424
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 577, time: 0.8124167919158936
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 578, time: 0.8437252044677734
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 579, time: 1.4217774868011475
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 580, time: 0.8280670642852783
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 581, time: 0.8280713558197021
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 582, time: 0.6093277931213379
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of batch: 583, time: 0.6405577659606934
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 58% |
1
2
end of ba

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 648, time: 0.5937113761901855
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 649, time: 0.6093597412109375
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 650, time: 0.8749415874481201
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 651, time: 0.3749518394470215
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 652, time: 0.49998950958251953
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 653, time: 0.6405837535858154
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 654, time: 0.5624628067016602
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 655, time: 0.4530937671661377
GPU Usage
| ID | GPU | MEM |
------------------
|  0 | 59% | 67% |
1
2
end of b

GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 720, time: 0.6358256340026855
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 721, time: 0.8294415473937988
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 722, time: 0.6893653869628906
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 723, time: 0.8489725589752197
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 724, time: 0.6030149459838867
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 725, time: 0.6458592414855957
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of batch: 726, time: 0.6955065727233887
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  1% | 67% |
1
2
end of batch: 727, time: 0.938793420791626
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 67% |
1
2
end of bat

KeyboardInterrupt: 

In [11]:
from joblib import Parallel, delayed

def chunker(iterable, total_length, chunksize):
    print("chunker")
    GPUtil.showUtilization()
    return (iterable[pos: pos + chunksize] for pos in range(0, total_length, chunksize))

def flatten(list_of_lists):
    "Flatten a list of lists to a combined list"
    return [item for sublist in list_of_lists for item in sublist]

def process_chunk(texts):
    preproc_pipe = []
    for doc in nlp.pipe(texts, batch_size=64):
        preproc_pipe.append(lemmatize_pipe(doc))
        print("spacy pipe ")
        GPUtil.showUtilization()

    return preproc_pipe

def preprocess_parallel(texts, chunksize=100):
    executor = Parallel(n_jobs=7, backend='multiprocessing', prefer="processes")
    print(1)
    do = delayed(process_chunk)
    print(2)
    tasks = (do(chunk) for chunk in chunker(texts, texts.shape[0], chunksize=chunksize))
    print(3)
    result = executor(tasks)
    return flatten(result)

In [ ]:
%%time
df['preproc_parallel'] = preprocess_parallel(df['clean'], chunksize=1024)

# AttributeError: Can't get attribute 'process_chunk' on <module '__main__' (built-in)>

1
2
chunker
| ID | GPU | MEM |
------------------
|  0 |  3% |  8% |
3


In [ ]:
# précédemment, lorsque je pensais pouvoir tout stocker dans une liste :

# df["lemmatized_text"] = lemma_text_list
# df["lemmatized_pos"] = lemma_pos_list
# df.to_csv("lemmatized_df.csv", index = False)

In [ ]:
# pour afficher l'ensemble des cells, à n'utiliser que pour afficher une portion seulement

# pd.set_option('display.max_colwidth', None)